# Transformers explained

In December 2017, Vaswani et al. published their seminal paper, Attention Is All You Need. They performed their work at Google Research and Google Brain. Lets look at the transformer model that is described in *Attention is All You Need*  


The transformer model is a stack of 6 layers. The output of layer $l$ is the input of layer $l+1$ until the final prediction is reached.There is a 6 layer encoder stack on left and a 6 layer decoder stack on the right.

![](data/Attention.png)

On the left, the inputs enter the encoder side of the Transformer through an **Attention** sub-layer and **FeedForward Network (FFN)** sub-layer. On the right, the target outputs go into the decoder side of the Transformer through two attention sub-layers and an FFN sub-layer.

The attention mechanism is a "word-to-word" operation. The attention mechanism will find how each word is related to all other words in a sequence, including the word being analyzed itself. Let's examine the following sequence:

*The cat sat on the mat.*

Attention will run dot products between word vectors and determine the strongest relationships of a word among all the other words, including itself. 
('cat' and 'cat').The attention mechanism will provide a deeper relationship between words and produce better results.For each attention sub-layer, the Transformer model runs not one but eight attention mechanisms in parallel to speed up the calculations. We shall discuss how these attention mechanism works in detail in the comnig sections.

## The encoder stack

The layers of the encoder and decoder of the original Transformer model are stacks of layers. Each layer of the encoder stack has the following structure:

![](data/encoder.png)

The original encoder layer structure remains the same for all of the N=6 layers of the Transformer model. Each layer contains two main sub-layers: a **multi-headed attention mechanism** and a **fully connected position-wise feedforward network**.  
Notice that a **residual connection** surrounds each **main sub-layer, Sublayer(x)**, in the Transformer model. These connections transport the unprocessed **input x** of a sublayer to a layer normalization function. This way, we are certain that key information such as **positional encoding** is not lost on the way. The normalized output of each
layer is thus: **LayerNormalization (x + Sublayer(x)).** Though the structure of each of the N=6 layers of the encoder is identical, the content of each layer is not strictly identical to the previous layer. Each layer learns from the previous layer and explores different ways of associating the tokens in the sequence.

The designers of the Transformer introduced a very efficient constraint. The output of every sub-layer of the model has a constant dimension, including the **embedding layer** and the **residual connections**. This dimension is $d_{model}$ and can be set to another value depending on your goals. In the original Transformer architecture, $d_{model}$ =512.

## Input embedding

The input embedding sub-layer converts the input tokens to vectors of dimension $d_{model}$ = 512 using learned embeddings in the original Transformer model. The structure of the input embedding is classical.

**Tokenizer:** 
A tokenizer normalizes the string to lower case and truncated it into subparts. A tokenizer will generally provide an integer representation that will be used for the embedding process.
```
Text = "The cat slept on the couch. It was too tired to get up."
Tokenized text= [1996, 4937, 7771, 2006, 1996, 6411, 1012, 2009, 2001, 2205, 5458, 2000, 2131, 2039, 1012]
```
There is not enough information in the tokenized text at this point to go further. The tokenized text must be embedded.
The Transformer contains a learned embedding sub-layer. Many embedding methods can be applied to the tokenized input.

**Word Embeddedings:**
Word embedding is a process of converting words  in to vector representations in a way that similar words have similar representations. Ther are many ways to do this like Bag of words, TF-IDF, Word2vec, Skip-Gram, Continuous Bag-of-words, Google Word2vec,Stanford Glove Embeddings etc..

Since we must produce a vector of size $d_{model}$ = 512 for each word, we will obtain a size 512 vector embedding for each word which looks something like this
```
word =[[ 1.35794589e-02 -2.18823571e-02 1.34526128e-02 6.74355254e-02
1.04376070e-01 1.09921647e-02 -5.46298288e-02 -1.18385479e-02
4.41223830e-02 -1.84863899e-02 -6.84073642e-02 3.21860164e-02
4.09143828e-02 -2.74433400e-02 -2.47369967e-02 7.74542615e-02
9.80964210e-03 2.94299088e-02 2.93895267e-02 -3.29437815e-02
…
7.20389187e-02 1.57317147e-02 -3.10291946e-02 -5.51304631e-02
-7.03861639e-02 7.40829483e-02 1.04319192e-02 -2.01565702e-03
2.43322570e-02 1.92969330e-02 2.57341694e-02 -1.13280728e-01
8.45847875e-02 4.90090018e-03 5.33546880e-02 -2.31553353e-02
3.87288055e-05 3.31782512e-02 -4.00604047e-02 -1.02028981e-01
3.49597558e-02 -1.71501152e-02 3.55573371e-02 -1.77437533e-02
-5.94457164e-02 2.21221056e-02 9.73121971e-02 -4.90022525e-02]]
```
Now that we have word embeddings of each word in sentence we need to look for positions of words in the sentence. Since we have word embeddings of dimensions 512. we need to add positional information to it with a dimension of 512.

Vaswani et al. provide sine and cosine functions that we can generate different frerquencies for the positional encoding (PE) for each position and each dimension i of the $d_{model}$ = 512 of the word embedding vector:

$$PE_{pos 2i} = \sin \bigg( \bigg)$$